In [1]:
# import neccessary libraries
import os
import sys
import json
import requests
from tqdm import tqdm
import time
import datetime
import logging
import logging.handlers
import torch
import pandas as pd
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os
from datasets import load_dataset
import torch.nn.parallel
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import matplotlib.pyplot as plt
import seaborn as sns

/home/mingxu/miniconda3/envs/ML/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def findFiles(path): return glob.glob(path)
print(findFiles('names/*.txt'))
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)
# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['names/Italian.txt', 'names/Japanese.txt', 'names/Greek.txt', 'names/Arabic.txt', 'names/French.txt', 'names/Polish.txt', 'names/Czech.txt', 'names/Vietnamese.txt', 'names/English.txt', 'names/Irish.txt', 'names/Scottish.txt', 'names/Korean.txt', 'names/Russian.txt', 'names/Portuguese.txt', 'names/Dutch.txt', 'names/Spanish.txt', 'names/Chinese.txt', 'names/German.txt']
Slusarski


Now we have category_lines, a dictionary mapping each category (language) to a list of lines (names). We also kept track of all_categories (just a list of languages) and n_categories for later reference.

In [3]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [35]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


In [5]:
import wandb
import random
wandb.init(
    project="DL_Lab3",
    config={
    "learning_rate": 5e-4,
    "architecture": "LSTM",
    "dataset": "NamedEntityRecognition",
    "liers": 2e4,
    "hidden_dim": 128,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mingxu_zhang (mingxus-team). Use `wandb login --relogin` to force relogin


In [46]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        # 直接实现 LSTM 而不调用 nn.LSTM
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Gates' weights and biases
        self.w_i = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.w_f = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.w_c = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.w_o = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.u_i = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.u_f = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.u_c = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.u_o = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_i = nn.Parameter(torch.Tensor(hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(hidden_size))
        self.b_c = nn.Parameter(torch.Tensor(hidden_size))
        self.b_o = nn.Parameter(torch.Tensor(hidden_size))

        self.hidden2out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)  # Ensure that 'dim=1' is correct

    def forward(self, input, hidden):
        # Assume 'input' is (seq_len, batch, input_dim)
        hx, cx = hidden
        for i in range(input.size(0)):  # Process each time step
            x = input[i]
            i_t = torch.sigmoid(x @ self.w_i + hx @ self.u_i + self.b_i)
            f_t = torch.sigmoid(x @ self.w_f + hx @ self.u_f + self.b_f)
            c_t = f_t * cx + i_t * torch.tanh(x @ self.w_c + hx @ self.u_c + self.b_c)
            o_t = torch.sigmoid(x @ self.w_o + hx @ self.u_o + self.b_o)
            hx = o_t * torch.tanh(c_t)
            cx = c_t
        output = self.hidden2out(hx)
        output = self.softmax(output)
        return output, (hx, cx)

    def init_hidden(self):
        return (torch.zeros(1, self.hidden_size),
                torch.zeros(1, self.hidden_size))

    def evaluate(self, test_data, all_categories, n_letters, all_letters):
        self.eval()  # Sets the module in evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():
            for category in all_categories:
                for line in test_data[category]:
                    category_index = all_categories.index(category)
                    line_tensor = lineToTensor(line)  # Properly convert line to tensor
                    line_tensor = line_tensor.unsqueeze(0)  # Add batch dimension
                    hidden = self.init_hidden()  # Initialize hidden state

                    output, hidden = self(line_tensor, hidden)  # Call forward pass
                    _, predicted = torch.max(output, 1)
                    total += 1
                    correct += (predicted == category_index).sum().item()
        accuracy = correct / total
        return accuracy


n_hidden = 128
rnn = LSTM(n_letters, n_hidden, n_categories)

In [38]:
def split_data(category_lines, split_ratio=0.8):
    train_data = {}
    test_data = {}

    for category, lines in category_lines.items():
        random.shuffle(lines)  # 打乱每个类别中的名字列表
        split_point = int(len(lines) * split_ratio)  # 计算划分点
        train_data[category] = lines[:split_point]
        test_data[category] = lines[split_point:]

    return train_data, test_data
train_data, test_data = split_data(category_lines, split_ratio=0.8)

In [39]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample(train_data, all_categories):
    category = randomChoice(list(train_data))  # Ensure to pass a list of categories
    line = randomChoice(train_data[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample(train_data, all_categories)
    print('category =', category, '/ line =', line)

category = Polish / line = Sokolowski
category = English / line = Hoare
category = Portuguese / line = Nunes
category = Chinese / line = Zhi
category = Arabic / line = Shadid
category = Spanish / line = Amador
category = Greek / line = Protopsaltis
category = Dutch / line = Schneider
category = French / line = Beaumont
category = Korean / line = Yun


In [63]:
criterion = nn.NLLLoss()
learning_rate = 5e-4 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    h0, c0 = rnn.initHidden()
    rnn.zero_grad()
    output, h, c = rnn(line_tensor, h0, c0)
    loss = criterion(output[-1], category_tensor)
    loss.backward()
    wandb.log({"loss": loss.item()})
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
    return output, loss.item()

    

In [64]:
n_iters = 10000
for iter in tqdm(range(1, n_iters + 1)):
    category, line, category_tensor, line_tensor = randomTrainingExample(train_data, all_categories)
    output, loss = train(category_tensor, line_tensor)

    # Add current loss avg to list of losses
    if iter % 500 == 0:
        print(f'Iteration {i+1}, Loss: {loss}')
        test_accuracy = rnn.evaluate(test_data, all_categories, n_letters, all_letters)
        wandb.log({"accuracy": test_accuracy})

  0%|          | 0/10000 [00:00<?, ?it/s]


RuntimeError: size mismatch (got input: [18], target: [1])

### Running on User Input

In [ ]:
import math
import numpy as np

def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (np.exp(value), all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('Dovesky')
predict('Jackson')
predict('Hou')


> Dovesky
(0.68) Russian
(0.25) Czech
(0.03) English

> Jackson
(0.70) Scottish
(0.18) English
(0.05) Russian

> Hou
(0.49) Chinese
(0.34) Korean
(0.10) Vietnamese
